## Air Quality Index: 5 Days Relative Mortality II

### Table of Content

<font size="3">

* [Environment setup](#cell2)
<br><br>
* [Mortality Risk calculation](#cell3)
<br><br>
* [Model setup](#cell4)
<br><br>
* [Training and Testing](#cell5)
<br><br>
* [Result Visualization & Conclusion](#cell6)

</font>

<a id="cell2"></a>

### Environment setup

In [133]:
#### Packages

import numpy as np
import pandas as pd
import time
import datetime
from statistics import mean
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import os
from math import sqrt

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def time_sep(df, col):
    df['year'] = df[col].dt.year
    df['month'] = df[col].dt.month
    df['day'] = df[col].dt.day
    df['hour'] = df[col].dt.hour
    df['weekday'] = df[col].dt.weekday
    return df


os.chdir('/Users/jerry/Desktop/') 
airdata = pd.read_csv('airdata_processed.csv', parse_dates = ['Datetime'], usecols = ['Datetime','PT08.S4(NO2)','T','RH','AH','adj_NO2 (μg/m3)'])
airdata = reduce_mem_usage(airdata)
airdata = time_sep(airdata, 'Datetime')

airdata.head(3)


Mem. usage decreased to  0.16 Mb (62.5% reduction)


,Datetime,PT08.S4(NO2),T,RH,AH,adj_NO2 (μg/m3),year,month,day,hour,weekday
0,2004-03-10 18:00:00,1692.0,13.601562,48.90625,0.757812,143.375,2004,3,10,18,2
1,2004-03-10 19:00:00,1559.0,13.296875,47.68750,0.725586,122.625,2004,3,10,19,2
2,2004-03-10 20:00:00,1555.0,11.898438,54.00000,0.750000,122.000,2004,3,10,20,2


<a id="cell3"></a>

### Calculate the Increase in mortality risk
<br>
From reference, we can take the baseline: 47 microg/m^3, which comes from the average of NO2 the mean in 10 city, to calculate mortality risk

In [134]:
if [(airdata['month'] >3) | (airdata['month'] <10)]:
    airdata['Increase in mortality risk(%)'] = (airdata['adj_NO2 (μg/m3)'] - 47) * 4.64 / 10
else:
    airdata['Increase in mortality risk(%)'] = (airdata['adj_NO2 (μg/m3)'] - 47) * 1.18 / 10

airdata['Increase in mortality risk(%)'] = airdata['Increase in mortality risk(%)'].astype('float64')
airdata = airdata.drop(['PT08.S4(NO2)', 'adj_NO2 (μg/m3)','Datetime'], axis = 1)
airdata.head(3)

,T,RH,AH,year,month,day,hour,weekday,Increase in mortality risk(%)
0,13.601562,48.90625,0.757812,2004,3,10,18,2,44.71875
1,13.296875,47.68750,0.725586,2004,3,10,19,2,35.09375
2,11.898438,54.00000,0.750000,2004,3,10,20,2,34.81250


----

<a id="cell4"></a>

### Model setup

With Lag = 8, based on the conclusion of previous data visualization.

In [135]:
## Parameter
lag = 8 ## this will be used in lag_producer
train_ratio = 0.8 ## this will be used in train_test_split

## Lagging shift and Splitting train & test
def lag_producer(df):
    for x in (range(1, lag+1)):
        df['lag' + str(x)] = df['Increase in mortality risk(%)'].shift(x)
    df = df.dropna(axis = 0)
    return df

def train_test_split(df):
    y = df['Increase in mortality risk(%)']
    x = df.loc[:, df.columns != 'Increase in mortality risk(%)']
    train_size = int(len(x) * train_ratio)
    x_train = x[0:train_size]
    y_train = x[train_size:len(x)]
    x_test, y_test = y[0:train_size], y[train_size:len(x)]
    return x_train, x_test, y_train, y_test

lag8 = lag_producer(airdata)
x_train, x_test, y_train, y_test = train_test_split(lag8)

display(x_train.head(5))
display(x_test.head(5))

final_ans = y_test.copy()

,T,RH,AH,year,month,day,hour,weekday,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8
8,10.703125,59.68750,0.764648,2004,3,11,2,3,18.671875,18.671875,23.031250,30.078125,36.906250,34.812500,35.093750,44.718750
9,10.296875,60.18750,0.751465,2004,3,11,3,3,14.562500,18.671875,18.671875,23.031250,30.078125,36.906250,34.812500,35.093750
10,10.101562,60.50000,0.746582,2004,3,11,4,3,11.578125,14.562500,18.671875,18.671875,23.031250,30.078125,36.906250,34.812500
11,11.000000,56.18750,0.736816,2004,3,11,5,3,8.820312,11.578125,14.562500,18.671875,18.671875,23.031250,30.078125,36.906250
12,10.500000,58.09375,0.735352,2004,3,11,6,3,7.730469,8.820312,11.578125,14.562500,18.671875,18.671875,23.031250,30.078125


8     14.562500
9     11.578125
10     8.820312
11     7.730469
12    10.554688
Name: Increase in mortality risk(%), dtype: float64

### Visualize Train and Test set

In [97]:
## Visualization of Train and Test
time = x_train["year"].map(str)+ '/' + x_train["month"].map(str) + '/'+  x_train["day"].map(str) +' ' + x_train['hour'].map(str) + ":00"
time1 = y_train["year"].map(str)+ '/' + y_train["month"].map(str) + '/'+  y_train["day"].map(str)+' ' + y_train['hour'].map(str) + ":00"

trace1 = go.Scatter(x = time,
                    y = x_test,
                    name = 'Train',
                    marker=dict(color='rgb(22, 96, 167)'))
trace2 = go.Scatter(x = time1,
                    y = y_test,
                    name = 'Test',
                    marker=dict(color='#851e52'))

layout = go.Layout(dict(title = "Increase in mortality risk on time series",
                        width=1000,
                        height=500,
                        xaxis = dict(range = ['2004-03','2005-04'], dtick =720.0, tickangle=45),
                        yaxis = dict(title = 'Mortality risk increase(%)', range = (-50, 130)
                        ),legend=dict(orientation="v")))

datastore = []
datastore.append(trace1)
datastore.append(trace2)

py.iplot(dict(data=datastore, layout=layout))
print('Proportion of Train to Test = ', train_ratio, ':', round(1-train_ratio,2))


Proportion of Train to Test =  0.8 : 0.2


<a id="cell5"></a>

### Training with Random forest (RF)

In [113]:
rf = RandomForestRegressor(n_estimators= 1000, 
                           min_samples_split= 10, 
                           min_samples_leaf= 4, 
                           max_features= 'auto', 
                           max_depth= 30,
                           random_state = 42)

rf.fit(x_train, x_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

<a id="cell6"></a>

### Test with RF and Visualize the outcome

In [132]:
outcomestore = rf.predict(y_train)

print('RMSE: ', sqrt(mean_squared_error(y_test, outcomestore)))

#### prediction plot and true value

time = y_train["year"].map(str)+ '/' + y_train["month"].map(str) + '/'+  y_train["day"].map(str) +' ' + y_train['hour'].map(str) + ":00"
temp1 = final_ans
temp2 = outcomestore

trace1 = go.Scatter(x = time,
                    y = temp1,
                    name = 'True value',
                    marker=dict(color='rgb(1, 1, 1)'))
trace2 = go.Scatter(x = time,
                    y = temp2,
                    name = 'Prediction',
                    marker=dict(color='#851e52'))

layout = go.Layout(dict(title = "Predict and True value on Testing data",
                        width=800,
                        height=500,
                        margin=go.layout.Margin(
                            l=50,
                            r=50,
                            b=180,
                            t=50,
                        ),
                        xaxis = dict(range = ['2004-03','2005-04'],  tick0 = 4.0, dtick =100.0, tickangle=45),
                        yaxis = dict(title = 'Mortality risk increase(%)', range = (-50, 100)
                        ),legend=dict(
    orientation="v")))

datastore = []
datastore.append(trace1)
datastore.append(trace2)

py.iplot(dict(data=datastore, layout=layout))


RMSE:  7.789147319056255


### Conclusion

- Different from the traditional time series model like ARIMA, machine learning methods provide a way to include other EXO variables into consideration.
<br>
- By using the Lagging method, it helps machine learning methods not to lose the sequential information in time series.
<br>
- The random forest show pretty good predicion accuracy with this data with RMSE = 7.78
<br>
- Large portion of RMSE are contributed within the date 2005. Feb. 28 to 2005. Mar. 02, it might because the insufficient information in the same period in 2004.